In [ ]:
---
layout: post
categories: [Overworld Lessons]
title: Overworld Level Profiles 
description:  Programmer Profiles 
type: issues 
comments: true
permalink: /overworld/EnemyCollision1
---

# Enemy Collisions

Our game is already filled with different types of collision reactions with NPCs. However, in order to make our game even more challenging, we can introduce the idea of having an Enemy. Having an enemy doesn't come without its consequences. In this lesson, you will learn how to create consequences when our player collides with an enemy. 

### Creating an Enemy
In order to make our enemy, we decided to create a new class for it. We made a file titled 'Enemy.js'. Like the other aspects of our game, this allows us to store all the functions for the enemy without making a mess of our code. We extended this class from our character class. There it is:

```js
import Character from './Character.js';
import Player from './Player.js';

class Enemy extends Character {
    constructor(data = null, gameEnv = null) {
        super(data, gameEnv);
        this.playerDestroyed = false; // Tracks if the player has been "killed"
    }

    // Overrides the update method to handle collision detection.
    update() {
        // Start by drawing the object
        this.draw();

        // Check if the enemy collides with the player
        if (!this.playerDestroyed && this.collisionChecks()) {
            this.handleCollisionEvent();
        }
    }

    // Checks if the Enemy collides with the Player.
    // Returns true if a collision is detected, otherwise false.
    collisionChecks() {
        for (const gameObj of this.gameEnv.gameObjects) {
            if (gameObj instanceof Player) {
                this.isCollision(gameObj);
                if (this.collisionData.hit) {
                    return true;
                }
            }
        }
        return false;
    }

    // Handles what happens when the player collides with the enemy.
    handleCollisionEvent() {
        console.log("Player collided with the Enemy. Player is dead.");
        this.playerDestroyed = true; // Mark the player as "dead"
        this.gameEnv.gameControl.restartLevel(); // Restart the level
    }
}

export default Enemy;
```

In this version, we have it so that the player dies and the game automatically resets after collision. There are no animations in this version--that's for the next lesson. 
